# Graph Neural Network
---
> Graph neural network model for vertex dynamics and tension prediction

**To-Do**👷🚧
- need a func-n w/ a **rollout error**,
- convert vel-y error to **position error**, e.g. "speed"+"direction"(angle/dot product etc.)
- *Training loop*:
- [ ] Write rollout prediction code.
- [ ] Experiments (**save all models** w/ backups; will need to test on the real tensions)<br>
*Params*: edge attrib-s, cell layer, arch(skip con-s), input noise (for long term prediction). *Errors*: 1-step, and rollout error (check after training at test time).
    1. Experiment with GN arch-s w/ *residual* (w/ skip) and *non-residual* architectures.
    1. *Edge directions experiment*: train w/ and w/o edge dir-s, do edge dir-s help to speed up training?
    1. *Cell layer*: does having dedicated cell processing layer help to increase the accuracy?
    1. For best (resnet or non-resnet) run *number of layers and dim-n sizes*. Try deep nets, how does accuracy change with increasing the depth of the net?
    1. If resnet is better: try w/and w/o edge dir-s and "cell layers".
    1. Number of previous velocities (window size).
    1. Rollout experiment 1— *input noise*: according to Sanchez-Gonzalez, *et al.* \[ASG2020\], Brownian noise improves rollout accuracies (long term accuracy of the whole movie, and/or prediction stability/robustness).
    1. Rollout experiment 2— *rollout training*: train directly on rollout, i.e. use 5- and 10-step loss instead of a 1-step loss for training (slower training).
    1. Optional:
        - compare MLP vs CONV layers for message passing.
        - try with dynamic graphs (construct graphs on the fly based on relative positions, and use cell edges and cell attrib only for queries on `Y_edge`, `Y_cell`).
- [ ] Ablation dataset (*real*).

**Node-to-Cell Encoding/Pooling Layer**:
1. Initiate node-to-cell edge attr-s as (source) node attr-s `x[node2cell_index[0]]`.
1. Compute node-to-cell edge attr-s using MLP: `e_n2c = MLP( x[node2cell_index[0]] )`
1. Aggregate node-to-cell edge attr-s as cell attr-s : `x_cell = Aggregate(e_n2c)`
1. Compute new cell attr-s using (encodes `x_cell` into cell attr-s) : `h_cell = MLP_Cell_encoder( x_cell )`

```python
n2c_model = mlp(...) # "message", just node-wise MLP
cell_aggr = Aggregate()
cell_enc = mlp(...)

e_n2c = n2c_model(data.x)[data.node2cell_index[0]]
x_cell = cell_aggr(data.cell_pressures.size(0), data.node2cell_index, e_n2c)
h_cell = cell_enc(x_cell)
```

---
- **Adding noise from M steps**: Sum of M normal rand. var-s results in normal var. w/ variance M and s.t.d.=sqrt(M):
```python
x = np.random.normal(size=(5,1000))
y = x.sum(axis=0)
z = np.random.normal(size=(1,1000))*np.sqrt(5)
plt.hist(x.ravel(),bins=50,label='x',density=True)
plt.hist(y        ,bins=50,label='y',density=True)
plt.hist(z.ravel(),bins=50,label='z',density=True,alpha=.5)
plt.legend();
```

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from os import path
import datetime

from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T

from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx
from simgnn.datautils import load_array, load_graph

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,10) # use larger for presentation
matplotlib.rcParams['font.size']= 16 # use 14 for presentation

In [3]:
from collections import OrderedDict
from simgnn.datasets import persistence_loss, VertexDynamics, HaraMovies, HaraAblation
from simgnn.transforms import Pos2Vec, ScaleVelocity, ScaleTension, ScalePressure, Reshape_x, TransformTension, RecoilAsTension
# from torch_geometric.utils import to_undirected as T_undir

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dtype = torch.float32
print(f'Defaults:\n |-device: {device}\n |-dtype : {dtype}')

Defaults:
 |-device: cuda
 |-dtype : torch.float32


## Data, Stat-s, Transforms

Notes for normalisation param-s:

- Use `simgnn.transforms` to normalise real--Hara movies and ablation,  and simulated movies. Use same normalisation constants for all simulated movies.
- For ablation movies, convert to  HaraMovies length scale (pixels) and use same `l_av` as for HaraMovies dataset.

---
|Dataset| Avg. edge len.|Tension/Recoil| Transforms (Normalisation)|Notes|
|:---:|---:|---:|:---|:---:|
|`single_distr_sims` | 1.08 a.u. | range=[0.00131, 4.26]; sd=0.637; median=0.305; mean=0.578; | `[Pos2Vec(scale=10*1.0)`, `ScaleVelocity(0.5*1.0)`, `ScaleTension(0.634,shift=0.6)]`| synthetic; w/o base tension |
|`unimodal_wbasetens_sims` | 0.906 a.u. | range=[0.407, 5.47]; sd=0.729; median=1.48; mean=1.6| `[Pos2Vec(scale=10*1.0)`, `ScaleVelocity(0.5*1.0)`, `ScaleTension(0.634,shift=0.6)]`| synthetic; w/ non-zero base tension |
|`HaraMovies`  | 26.32 pixels||`[Pos2Vec(scale=10*26.32)`, `ScaleVelocity(0.5*26.32)]`| tissue movies w/o force data|
|`HaraAblation`|$26.32\cdot\frac{0.4}{0.21}$ pixels| range=[0.0239, 2.61]; sd=0.651; median=0.673; mean=0.826 | `[Pos2Vec(scale=10*50.13), ScaleVelocity(0.5*50.13)]`| short movies w/ recoil data; frame rate:dataset contains vertex positions only from every 10th frame |

- Logarithm of tension: ln(Tension/Recoil {raw values})
```
Single_distr_sims:        range: [-6.63 1.45]; s.d.: 1.22 || median: -1.19; mean: -1.2;
Unimodal_wbasetens_sims : range: [-0.9  1.7]; s.d.: 0.451 || median: 0.39; mean: 0.371;
Hara ablation:            range: [-3.73 0.958]; s.d.:0.962 || median:-0.396; mean:-0.547;
```


---
- **Normalisation param-s**

In [5]:
# Data normalisation param-s : sim="default" for simulation data
# Average edge lengths
l_0 = {'sim':1.0, 'single_distr_sims':1.1, 'unimodal_wbasetens_sims':0.91, 'hara':26.32, 'abln':50.13}

# Stat-s for raw tension/recoil values
t_av = {'sim':0.6, 'single_distr_sims': 0.58, 'unimodal_wbasetens_sims': 1.6, 'abln': 0.83}
t_sd = {'sim':0.63, 'single_distr_sims': 0.64, 'unimodal_wbasetens_sims':0.73 , 'abln': 0.65}

### Datasets

In [98]:
!rm -dr simgnn_data/single_distr_sims/train/processed
!rm -dr simgnn_data/single_distr_sims/val/processed
!rm -dr simgnn_data/unimodal_wbasetens_sims/train/processed
!rm -dr simgnn_data/unimodal_wbasetens_sims/val/processed

!rm -dr simgnn_data/hara_movies/processed/
!rm -dr simgnn_data/hara_ablation/processed/

# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg001/processed/
# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg003/processed/
# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg005/processed/

**Input features and normalisation param-s (transforms)**

In [99]:
# Input features
window_size = 5
print(f'> Set features window size (#past velocities) to {window_size}')

# Normalisation
# for simulated data (simul params ~ normal distr-s): l0_sim=1.0, l0_HaraMovie: l0=26.32
Tnrm = {} # dict of transforms

# Default for all simulations:
Tnrm['sim'] = T.Compose([Pos2Vec(scale=10*l_0['sim']) , ScaleVelocity(0.5*l_0['single_distr_sims']),
                         ScaleTension(t_sd['sim'], shift=t_av['sim']),
                         Reshape_x((-1,window_size*2)) ] )

# w/o base contractility:
Tnrm['single_distr_sims'] = T.Compose([Pos2Vec(scale=10*l_0['single_distr_sims']) , ScaleVelocity(0.5*l_0['single_distr_sims']),
                                       ScaleTension(t_sd['single_distr_sims'], shift=t_av['single_distr_sims']),
                                       Reshape_x((-1,window_size*2)) ] )

# w/ base contractility:
Tnrm['unimodal_wbasetens_sims'] = T.Compose([Pos2Vec(scale=10*l_0['unimodal_wbasetens_sims']) , ScaleVelocity(0.5*l_0['unimodal_wbasetens_sims']),
                                             ScaleTension(t_sd['unimodal_wbasetens_sims'], shift=t_av['unimodal_wbasetens_sims']),
                                             Reshape_x((-1,window_size*2)) ] )

# Hara ablation norm
Tnrm['abln'] = T.Compose([Pos2Vec(scale=10*l_0['abln']), ScaleVelocity(0.5*l_0['abln']), RecoilAsTension(),
                          ScaleTension(t_sd['abln'], shift=t_av['abln']),
                          Reshape_x((-1,window_size*2))]) 

# Hara movie dataset norm
Tnrm['hara'] = T.Compose([Pos2Vec(scale=10*l_0['hara']) , ScaleVelocity(0.5*l_0['hara']), Reshape_x((-1,window_size*2))]) 

> Set features window size (#past velocities) to 5


**Load datasets**

In [112]:
hara_abln = HaraAblation('simgnn_data/hara_ablation/', window_size=window_size, transform=Tnrm['abln'], smoothing=True, sma_lag_time=3)
hara = HaraMovies('simgnn_data/hara_movies/',window_size=window_size, transform=Tnrm['hara'], smoothing=True, sma_lag_time=4)

# Hara movie datasets
# seg001 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg001/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)
# seg001_raw = HaraMovies('simgnn_data/hara_movies/', window_size=window_size, transform=Tnorm_hara)
# seg003 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg003/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)
# seg005 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg005/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)

# Simulation datasets
# rand base contractility
sim1 = VertexDynamics('./simgnn_data/unimodal_wbasetens_sims/train/', window_size=window_size,
                      transform=Tnrm['sim']
#                       transform=Tnrm['unimodal_wbasetens_sims']
                     )
sim1_val = VertexDynamics('./simgnn_data/unimodal_wbasetens_sims/val/', window_size=window_size,
                          transform=Tnrm['sim']
#                           transform=Tnrm['unimodal_wbasetens_sims']
                         )

# w/o base contractility
sim2 = VertexDynamics('./simgnn_data/single_distr_sims/train/', window_size=window_size,
                      transform=Tnrm['sim']
#                       transform=Tnrm['single_distr_sims']
                     )
sim2_val = VertexDynamics('./simgnn_data/single_distr_sims/val/', window_size=window_size,
                          transform=Tnrm['sim']
#                           transform=Tnrm['single_distr_sims']
                         )


datasets_dict = {'train': sim1 + sim2,
                 'val': sim1_val,
                 'val2': sim2_val,
                 'hara' : hara,
                 'abln': hara_abln}
dataset_legend={'train': 'Train(sim1+sim2)',
                'val': 'Val(sim1)',
                'val2': 'Val(sim2)',
                'hara': 'Hara',
                'abln': 'Recoil'}

In [113]:
# k = 'val2'
# t = 15
# print('{}\n{} : {}\nsize:{}\n\nFrame {}:\n{}'.format(dataset_legend[k],k,datasets_dict[k],len(datasets_dict[k]),t,datasets_dict[k][0]))
# plt.figure(figsize=[2.5,2.5])
# nx.draw(to_networkx(datasets_dict[k][t]),pos=dict(enumerate(datasets_dict[k][t].pos.numpy())), node_size=10)

**Persistence Losses**

In [114]:
# print('Persistence:')
# for k in datasets_dict:
#     if k=='abln':print('Ignoring ablation data velocity');continue
#     print(f'\t- {dataset_legend[k]}: {persistence_loss(datasets_dict[k])}')
    
# print(len(datasets_dict['train']))

---

## Tension Models

In [115]:
from simgnn.train import train_model, write_log, load_log, plot_losses
from simgnn.train import predict, predict_batch, plot_velocity_predictions, plot_tension_prediction

from scipy.stats import pearsonr

# from simgnn.nn import mlp, IndependentBlock, MessageBlock, dims_to_dict, Encode_Process_Decode
from simgnn.models import construct_simple_gnn, GraphEncoder, GraphDecoder, GraphProcessor

In [116]:
# MessageBlock(..., hidden_dims=[16], updt='p')

In [186]:
torch.manual_seed(2)

# GNN architecture (if dim is `int` latent dims nodes=edges)
# -----
latent_dims = 128

encoder_kwrgs    = {'hidden_dims':[64]}

processor_kwargs = {'n_blocks': 20,
                    'block_type': 'message',
                    'is_residual': True,
                    'seq': 'n',
                    'norm_type': 'ln',
                    'block_p': 0.1,  # block dropout (last layer)
                    'dropout_p': 0, # dropout for hidden layers (if any)
                    'hidden_dims':[]
                   }

decoder_kwargs   = {'hidden_dims':[64, 32, 8]}

net = construct_simple_gnn(OrderedDict([('node', window_size*2), ('edge', 2)]), latent_dims, OrderedDict([('node', 2), ('edge', 1)]),
                           encoder_kwrgs=encoder_kwrgs, processor_kwargs=processor_kwargs, decoder_kwargs=decoder_kwargs).to(device)

# Training param-s & Data loaders:
# -----
num_epochs = 100
lr = 0.0001
bs = 2
loss_func = torch.nn.MSELoss(reduction='mean')
# loss_func = torch.nn.L1Loss(reduction='mean')

opt = torch.optim.Adam(net.parameters(), lr=lr) # SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = None # torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')

# in order to track the batch id for var-s add its key to "follow_batch":
loaders = {'train' : DataLoader(datasets_dict['train'], batch_size=bs, shuffle=True),
           'val':    DataLoader(datasets_dict['val'],  batch_size=bs),
           'val2':   DataLoader(datasets_dict['val2'], batch_size=bs),
#            'abln':   DataLoader(datasets_dict['abln'], batch_size=bs),
#            'hara':   DataLoader(datasets_dict['hara'], batch_size=bs)
          }
# this tracks batch id for "cell_pressures_batch" and "edge_index_batch" in addition to node batch ids

model_states = list(loaders.keys())
use_force_loss = {'train':[True,False], 
                  'val':[True, False],
                  'val2':[True,False] ,
                  'abln':[True,False] ,
                  'hara' : [False, False]
                 }# [tension, pressure]

return_best = False

model_name = datetime.datetime.now().strftime('%d%m%Y_%H%M%S')+f'_{net.__class__.__name__}_dp{processor_kwargs["dropout_p"]}'+\
             f'blcdp{processor_kwargs["block_p"]}bs{bs}w{window_size}{"best" if return_best else ""}'
save_dir = './simgnn_data/saved_models/'

In [ ]:
print(model_name,end='\n---\n')

net, train_log = train_model(net, loaders, opt, num_epochs=num_epochs, scheduler=scheduler, device=device, model_states = model_states,
                               loss_func = loss_func, use_force_loss=use_force_loss, return_best=return_best)

12102021_184818_Encode_Process_Decode_dp0blcdp0.1bs2w5
---
Training param-s: #epochs=100, metric=MSELoss(), batch_size=2, optim=Adam, sch-r=none, return_best=False
---
Epoch 0/99: train_loss_tot=   1.779; train_loss_y=0.008583; train_loss_T=   1.769; |val_loss_tot=    1.72; |val2_loss_tot=   1.811; |18s
Epoch 1/99: train_loss_tot=   1.479; train_loss_y=  0.0071; train_loss_T=   1.471; |val_loss_tot=   1.346; |val2_loss_tot=   1.161; |36s
Epoch 2/99: train_loss_tot=   1.281; train_loss_y=0.004967; train_loss_T=   1.276; |val_loss_tot=   1.259; |val2_loss_tot=  0.9951; |53s
Epoch 3/99: train_loss_tot=   1.079; train_loss_y=0.004101; train_loss_T=   1.075; |val_loss_tot=   1.082; |val2_loss_tot=  0.7549; |70s
Epoch 4/99: train_loss_tot=  0.9618; train_loss_y=0.003873; train_loss_T=  0.9575; |val_loss_tot=  0.6683; |val2_loss_tot=   1.014; |87s
Epoch 5/99: train_loss_tot=  0.8259; train_loss_y=0.003453; train_loss_T=   0.822; |val_loss_tot=  0.6099; |val2_loss_tot=   1.213; |105s
Epoch 6/9

In [ ]:
# plot losses
plot_losses(train_log, loaders, dataset_legend)
# plot_losses(train_log_ln_bs32, loaders, dataset_legend, marker='s', ms=15, linestyle='-', fillstyle='none', legend_prefix='[LN:bs=32] ', figsize=None)
# plt.show()

In [ ]:
# compute predictions and plot them
# vel-y, tens, pres
pred_out, tgt_out, sample_losses = predict_batch(net,
                                                 {
#                                                   'train' : DataLoader(datasets_dict['train'], batch_size=bs),
                                                  'val':    DataLoader(datasets_dict['val'],  batch_size=bs),
                                                  'val2':   DataLoader(datasets_dict['val2'], batch_size=bs),
                                                  'abln':   DataLoader(datasets_dict['abln'], batch_size=bs),
                                                  'hara':   DataLoader(datasets_dict['hara'], batch_size=bs)},
                                                 loss_func=torch.nn.MSELoss(reduction='mean'),
                                                 use_force_loss = use_force_loss, return_losses = True, device=device)
print(''.join([f'- {k} ={sample_losses[k]};\n' for k in sample_losses]))

plot_velocity_predictions(pred_out[0], tgt_out[0], dataset_legend)

plot_tension_prediction(pred_out[1], tgt_out[1], dataset_legend,figsize=[15,15],nrows=2,ncols=2)

if 'abln' in tgt_out[1]:
    y_gt = torch.cat(tgt_out[1]['abln'],axis=0).numpy()
    y_pred = torch.cat(pred_out[1]['abln'],axis=0).numpy()[~np.isnan(y_gt)]
    print(f'R,p={pearsonr(y_gt[~np.isnan(y_gt)], y_pred)}')